# Requirments



In [ ]:
"""
# get majka database
!curl --remote-name-all https://nlp.fi.muni.cz/ma{/majka.w-lt}
!mv majka.w-lt drive/MyDrive/data/
# download czech squad
!curl --remote-name-all https://lindat.cz/repository/xmlui/bitstream/handle/11234/1-3069{/sqad_v3.tar.xz}
!mv sqad_v3.tar.xz drive/MyDrive/data/
!tar -xf drive/MyDrive/data/sqad_v3.tar.xz
"""

In [ ]:
!pip install sentencepiece
!pip install datasets transformers

!pip install googletrans==4.0.0-rc1
!pip install wikipedia
!pip install rank_bm25
!pip install majka
!pip install corpy

!pip install tensorflow-gpu==1.15.2
!pip install deeppavlov
!pip install git+https://github.com/deepmipt/bert.git@feat/multi_gpu

Importing important libraries

In [ ]:
import torch
import string
import os
import sys
import time
import shutil
import json
import numpy as np
import collections
import datetime
from tqdm.auto import tqdm
import warnings
import json
import nltk.data
import nltk

from datasets import load_dataset, load_metric
from typing import List, Tuple, Dict
from collections import defaultdict
from transformers import AlbertTokenizerFast, AlbertForQuestionAnswering
from transformers import BertTokenizerFast, BertForQuestionAnswering

from deeppavlov import build_model, configs

from rank_bm25 import BM25Okapi, BM25Plus, BM25L
import re
import majka
from corpy.morphodita import Tagger
import wikipedia
from googletrans import Translator
import requests

from google.colab import drive

warnings.filterwarnings("ignore")

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Mount google drive 

In [ ]:
# Remove pre-cached sample data in colab's directory
if os.path.isdir("sample_data"):
  shutil.rmtree("sample_data")
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Get downloaded ner_model from drive

In [ ]:
!mkdir /root/.deeppavlov
!mkdir /root/.deeppavlov/models/
!mkdir /root/.deeppavlov/downloads/

!cp -r ./drive/MyDrive/ner_models/bert_models /root/.deeppavlov/downloads/
!cp -r ./drive/MyDrive/ner_models/ner_ontonotes_bert_mult /root/.deeppavlov/models/

!ls /root/.deeppavlov/downloads
!ls /root/.deeppavlov/models

Download and extract morphoita tagger model for lemmatization

In [ ]:
!curl --remote-name-all https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-1836{/czech-morfflex-pdt-161115.zip}
!unzip ./czech-morfflex-pdt-161115.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 69.1M  100 69.1M    0     0  32.1M      0  0:00:02  0:00:02 --:--:-- 32.1M
Archive:  ./czech-morfflex-pdt-161115.zip
   creating: czech-morfflex-pdt-161115/
  inflating: czech-morfflex-pdt-161115/README.html  
  inflating: czech-morfflex-pdt-161115/README  
  inflating: czech-morfflex-pdt-161115/czech-morfflex-pdt-161115-no_dia-pos_only.tagger  
  inflating: czech-morfflex-pdt-161115/czech-morfflex-pdt-161115-pos_only.tagger  
  inflating: czech-morfflex-pdt-161115/czech-morfflex-161115-no_dia.dict  
  inflating: czech-morfflex-pdt-161115/czech-morfflex-pdt-161115.tagger  
  inflating: czech-morfflex-pdt-161115/czech-morfflex-161115-no_dia-pos_only.dict  
  inflating: czech-morfflex-pdt-161115/czech-morfflex-161115.dict  
  inflating: czech-morfflex-pdt-161115/LICENSE  
  inflating: czech-morfflex-pdt-161115/czech-morfflex-161

# Reader

In [ ]:
class Reader():

  def __init__(self, model_checkpoint, model_type="albert", max_answer_length=10, n_best_size=10, max_length=384, stride=128, use_cpu=False):
    # load all parameters of the reader
    self.max_answer_length = max_answer_length  # max answer span length
    self.n_best_size = n_best_size  # 
    self.max_length = max_length  # max count of tokens in one tokenized passage
    self.stride = stride  # the length of overlap between two mini-batches of tokenizer

    # choose device; cuda if available
    self.device = torch.device("cuda:0" if (torch.cuda.is_available() and use_cpu == False) else "cpu")

    if model_type == 'albert':
      # load tokenizer and model from pretrained checkpoint
      self.tokenizer = AlbertTokenizerFast.from_pretrained(model_checkpoint)
      # load model to device if possible
      self.model = AlbertForQuestionAnswering.from_pretrained(model_checkpoint).to(self.device)
    elif model_type == 'mbert':
      # load tokenizer and model from pretrained checkpoint
      self.tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)
      # load model to device if possible
      self.model = BertForQuestionAnswering.from_pretrained(model_checkpoint).to(self.device)
    else:
      print("Wrong model type parameter.")
      return None

    print("Model loaded from: " + model_checkpoint)
    print(f"Model has {self.count_parameters(self.model)} parameters")
    print("Device selected:")
    print(self.device)


  def decode(self, output, context, offset_mappings):
    """
    get the text span from the unnormalized log probabilities

    method has been partly borrowed from 
    https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/question_answering.ipynb

    """
    # enumerate over all outputs (max output size is 500 tokens in a log prob tensor)
    valid_answers = []

    for i, _ in enumerate(output.start_logits):

      start_logits = output.start_logits[i].cpu().detach().numpy()
      end_logits = output.end_logits[i].cpu().detach().numpy()
      offset_mapping = offset_mappings[i]

      # Gather the indices the best start/end logits:
      start_indexes = np.argsort(start_logits)[-1 : - self.n_best_size - 1 : -1].tolist()
      end_indexes = np.argsort(end_logits)[-1 : - self.n_best_size - 1 : -1].tolist()
      for start_index in start_indexes:
          for end_index in end_indexes:
              # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
              # to part of the input_ids that are not in the context.
              if (
                  start_index >= len(offset_mapping)
                  or end_index >= len(offset_mapping)
                  or offset_mapping[start_index] is None
                  or offset_mapping[end_index] is None
              ):
                  continue
              # Don't consider answers with a length that is either < 0 or > max_answer_length.
              if end_index < start_index or end_index - start_index + 1 > self.max_answer_length:
                  continue
              if start_index <= end_index: # We need to refine that test to check the answer is inside the context
                  start_char = offset_mapping[start_index][0]
                  end_char = offset_mapping[end_index][1]
                  valid_answers.append(
                      {
                          "score": start_logits[start_index] + end_logits[end_index],
                          "text": context[start_char: end_char].strip()
                      }
                  )
    valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:self.n_best_size]
    return valid_answers


  def get_answers(self, question, context):
    """
    get the best answers from the context to the question 

    """
    inputs = self.tokenizer(question, context, 
                      return_tensors='pt',
                      truncation="only_second",
                      max_length=self.max_length, # to prevent cuda running out of memory
                      stride=self.stride,     # overlap within splitted long
                      return_offsets_mapping=True,
                      return_overflowing_tokens=True,
                      padding="max_length")
    inputs.to(self.device)

    outputs = self.model(inputs['input_ids'], 
                    token_type_ids=inputs['token_type_ids'],
                    attention_mask=inputs['attention_mask'])
    
    valid_answers = self.decode(outputs, context, inputs['offset_mapping'])
    
    return valid_answers


  def count_parameters(self, model):
    """
    Counts the parameters of the model

    """

    return sum(p.numel() for p in self.model.parameters() if p.requires_grad)
    

# Retriever


In [ ]:
class Retriever():

  def __init__(self):
    wikipedia.set_lang("cs") 

    # save the most common czech words (stop words)
    common = "kdy být a se v na ten on že s z který mít do já o k i jeho ale svůj jako za moci pro tak po tento co když všechen už jak aby od nebo říci jeden jen můj jenž ty stát u muset chtít také až než ještě při jít pak před však ani vědět hodně podle další celý jiný mezi dát tady tam kde každý takový protože nic něco ne sám bez či dostat nějaký proto"
    self.common = common.split()

    # save punctuation to be removed
    punctuation = ". , ? ! ... \" ( ) ; - /"
    self.punctuation = punctuation.split()

    # majka lemmatizer settings
    self.morph = majka.Majka('drive/MyDrive/data/majka.w-lt')
    self.morph.flags = 0  # unset all flags
    self.morph.tags = False  # return just the lemma, do not process the tags
    self.morph.first_only = True  # return only the first entry
    self.morph.negative = "ne"

    # morphodita lemmatizer
    self.tagger = Tagger("./czech-morfflex-pdt-161115/czech-morfflex-pdt-161115.tagger")

    # load wiki titles and build index for search
    self.bm25_articles_index, self.titles = self.get_title_search_index()

    # load tokenizer to split text into sentences
    self.tokenizer = nltk.data.load('tokenizers/punkt/czech.pickle')

    # Download and load model (set download=False to skip download phase)
    self.ner = build_model(configs.ner.ner_ontonotes_bert_mult, download=False)


  def get_title_search_index(self):
    """
    Build index for searching through relevant title names on czech wiki:

    """

    f = open("drive/MyDrive/data/wiki/cswiki-latest-all-titles-in-ns0", "r")
    titles = []

    for line in f: 
      title = ((" ").join(line.split("_"))).strip()
      title = title.strip('\n')
      titles.append(title)

    f.close()

    # tokenize for bm25
    tok_titles = []
    for title in titles:
      tok_tit = re.split(" ", title.lower())
      for tok in tok_tit:
        if tok == "":
          tok_tit.remove("")
      tok_titles.append(tok_tit)

    bm25 = BM25Okapi(tok_titles)

    return bm25, titles


  def search_titles(self, question):
    """
    Search with bm25 among the wiki titles

    """
    tokenized_query = self.delete_common(self.lemmatize_morphodita(question.lower()))
    # print(tokenized_query)
    results = self.bm25_articles_index.get_top_n(tokenized_query, self.titles, n=5)

    return results


  def get_named_entities(self, question):
    """
    Extracts named entities from the question.

    """

    URL = "https://nlp.fi.muni.cz/projekty/ner/nerJSON.py"
    text = question
    PARAMS = {'text': text}
    r = requests.get(url = URL, params = PARAMS)
    data = r.json() 

    NEs = []

    if data != {}:
      for item in data:
        lemma = data[item]['lemma']
        NEs.append(lemma)

    return NEs

  def get_pavlov_named_entities(self, question):
    """
    Extracts named entities from the question.

    """
    ner_tags = self.ner([question])

    named_entities = ""
    for idx, tag in enumerate(ner_tags[1][0]):
      if tag != 'O':
        named_entities += ner_tags[0][0][idx] + " "

    NEs = []
    if len(named_entities.strip()) != 0:
      NEs.append(named_entities.strip())

    return NEs
  
  def iscommon(self, x):
    """
    decides if query token is common

    """
    if x in self.common or x in self.punctuation:
      return True
    else:
      return False


  def delete_common(self, tokens):
    """
    Remove the most common czech words from the query tokens (low information value)

    """
    tokens = [x for x in tokens if not self.iscommon(x)]
        
    return tokens

  
  def lemmatize(self, text):
    """
    Returns lemma of each token in a list of lemmatized tokens

    """

    tok_text = re.split("\W", text)

    # lemmatize each token
    lemmatized_tokens = []
    for token in tok_text:
      if token == '':
        continue
      lemma = self.morph.find(token)
      if len(lemma) == 0:
        lemmatized_tokens.append(token)
      else:
        lemmatized_tokens.append(lemma[0]['lemma'])

    return lemmatized_tokens

  def lemmatize_morphodita(self, text):
    """
    Returns lemma of each token in a list of lemmatized tokens

    """

    text = re.split("\W", text)
    text = (" ").join(text)

    tokens = list(self.tagger.tag(text, convert='strip_lemma_id'))

    lemmas = []
    for token in tokens:
      lemmas.append(token.lemma)

    return lemmas


  def search_again(self, tokens):
    """
    Performs repeated search in case wiki api didnt find any documents

    """
    searched_term = (' ').join(tokens)
    #print(searched_term)
    doc_list = wikipedia.search(searched_term, results=1)

    if len(tokens) == 0:
      return []

    if len(doc_list) == 0:
      del tokens[0]
      return self.search_again(tokens)

    return doc_list


  def get_doc_list(self, question):
    """
    Returns top 1-3 wiki arcitles that might answer the question topic

    """

    # get names entities if present
    named_ERs = self.get_pavlov_named_entities(question)
    # get relevant article title names
    relevant_titles = self.search_titles(question)

    #search for documents
    max_docs = 1
    doc_list = []

    # search based on recognised named entity
    if len(named_ERs) > 0:
      article = wikipedia.search(named_ERs[0], results=max_docs)
      if len(article) > 0:
        doc_list.append(article[0])
    # search based on best wiki title match
    if len(relevant_titles) > 0:
      article = wikipedia.search(relevant_titles[0], results=max_docs)
      if len(article) > 0:
        doc_list.append(article[0])

    # basic search for the question
    article = wikipedia.search(question, results=max_docs)
    # simplify the search if its too bad
    if len(article) == 0:
      # extract important for wiki
      tokens = self.delete_common(self.lemmatize_morphodita(question.lower()))
      article = self.search_again(tokens)
    doc_list.append(article[0])

    return doc_list

  
  def normalize_length(self, par):
    """
    Splits too long paragraph into smaller ones

    """

    #split long paragraph into sentences
    sentences = self.tokenizer.tokenize(par)

    normalized_pars = []
    new_paragraph = ""

    # iterate over sentences
    for idx, sentence in enumerate(sentences):
      
      if len(new_paragraph) + len(sentence) > 1500:
        normalized_pars.append(new_paragraph)
        new_paragraph = ""

        # make some overlap
        for k, trailing in enumerate(sentences[idx-2:idx]):
          new_paragraph += trailing

      else:
        new_paragraph += sentence
    
    return normalized_pars

  
  def split_documents(self, doc_list):
    """
    Splits each retrievede wiki article into paragraphs and normalizes its lengths

    """

    pars = []
    lemm_pars = []

    # iterate over articles and process each one
    for doc in doc_list:
      # get whole page content
      try:
        doc = wikipedia.page(doc)
      except wikipedia.DisambiguationError as e:
        s = e.options[0]
        try:
          doc = wikipedia.page(s)
        except wikipedia.DisambiguationError:
          continue

      result = re.split('=== Reference ===|== Reference ==', doc.content)[0]
      # split article into paragraphs
      result = re.split('== .*. ==|\\n\\n', result)

      # save stripped paragraphs
      for par in result:
        par = ((((par.strip()).strip('=')).strip('\n')).strip('\n\n')).strip('\r\n')

        # remove some trash
        if par == '' or par == '\n' or par.strip().startswith("Obrázky, zvuky či videa k tématu"):
          continue

        # check max paragraph length
        if len(par) > 1500:
          # split into smaller paragraphs
          normalized_paragraphs = self.normalize_length(par)
          # append each smaller paragraph
          for norm_par in normalized_paragraphs:
            pars.append(norm_par)
            lemm_pars.append((' ').join(self.delete_common(self.lemmatize_morphodita(norm_par.lower()))))
        else:
          # append paragraph
          pars.append(par)

          # get lemmas and append
          lemm_pars.append((' ').join(self.delete_common(self.lemmatize_morphodita(par.lower()))))

    return pars, lemm_pars

  def retrieve(self, question):  
    """
    Returns the top 3 paragraphs for the given question

    """
    # max question length
    if len(question) > 250:
      return ""
    # strip questionmark
    question = question.strip('?')

    # TODO SPEED THIS UP
    # get wiki documents
    doc_list = self.get_doc_list(question)

    # convert to set to only work with unique article names
    doc_list = set(doc_list)

    # TODO SPEED THIS UP
    # split docs into paragraphs
    pars, lemm_pars = self.split_documents(doc_list)

    # tokenize for bm25
    tok_text = []
    for par in lemm_pars:
      tok_par = re.split("\W", par)
      for tok in tok_par:
        if tok == "":
          tok_par.remove("")
      tok_text.append(tok_par)

    # build index
    bm25 = BM25Plus(tok_text)
    # bm25 = BM25Okapi(tok_text)

    # tokenize and lemmatize the query
    tokenized_query = (' ').join(self.delete_common(self.lemmatize_morphodita(question.lower())))
    tokenized_query = re.split("\W", tokenized_query)

    # get results
    results = bm25.get_top_n(tokenized_query, pars, n=3)

    return results, doc_list


  @staticmethod
  def count_log_conf(best_answer, all_answers):
    """
    Returns the sum of log probs 
    
    """

    log_conf = 0
    for answer in all_answers:
      if (best_answer in answer['text']) or (answer['text'] in best_answer):
        log_conf += answer['score']
    
    return log_conf


# Preprocessing SQAD

In [ ]:
class SqadDataset():

  def __init__(self, sqad_dir, save_dir="./sqad_processed", process_boolean=False):
    self.save_dir = save_dir
    self.sqad_dir = sqad_dir
    self.process_boolean = process_boolean

  def extract_answer(self, dirnum):
    """
    Parse the answer of current dataset record.
    Returns the parsed answer and its lemma.

    """

    f = open(f"{self.sqad_dir}/{dirnum}/09answer_extraction.vert", "r")

    q = f.read().split("\n")
    answer = ""
    answer_lemma = ""

    # parse answer
    for line in q:
      # split into columns
      line = line.split("\t")

      # end sign
      if line[0] == "</s>":
        break
      
      # get answer and its lemma
      line_a = line[0]
      if len(line) > 1:
        line_a_lemma = line[1]
        if line_a_lemma == "[number]":
          line_a_lemma = line[0]

      # process special signs
      if line_a in {"<s>", "<g/>", "</s>"}:
        answer = answer[:-1]
        answer_lemma = answer_lemma[:-1]
        continue

      # append answer
      answer += line_a + " "
      answer_lemma += line_a_lemma + " "
    
    f.close()
    return answer, answer_lemma


  def extract_question(self, dirnum):
    """
    Parse the answer of current dataset record.
    Returns the parsed answer and its lemma.
    
    """

    f = open(f"{self.sqad_dir}/{dirnum}/01question.vert", "r")

    q = f.read().split("\n")
    question = ""

    for line in q:
      line = line.split("\t")
      # end sign
      if line[0] == "</s>" and question[-1] == "?":
        break

      line = line[0]
      if line in {"<s>", "<g/>", "</s>"}:
        question = question[:-1]
        continue
      
      if line != "?":
        question += line + " "
      else:
        question += "? "

    f.close()
    return question


  def process_dataset(self, from_q, to_q):
    """
    Process the questions and answers from the sqad dataset and save it as a json file

    Process the dataset from record from_q to record to_q
    
    """

    sqad_dataset = {}
    counter = from_q

    for i in tqdm(range(from_q, to_q+1)):

      # get question number
      q_number = ""
      for _ in range(len(str(i)), 6):
        q_number += "0"
      q_number += str(i)

      # extract from dataset
      question = self.extract_question(q_number)
      correct_answer, lemmatized_answer = self.extract_answer(q_number)

      # exclude yes/no questions
      if not self.process_boolean:
        if correct_answer.strip() == "ano" or correct_answer.strip() == "ne":
          continue

      # save data
      data = {}
      data["question"] = question
      data["answer"] = correct_answer
      data["answer_lemma"] = lemmatized_answer

      sqad_dataset[counter] = data
      counter += 1

    # save extracted data as json
    with open(self.save_dir, "w") as f:
      json.dump(sqad_dataset, f)
      print("Sqad dataset has been processed to: " + self.save_dir)


  @staticmethod
  def load_sqad(saved_dataset_file):
    """
    Loads the saved json daataset as a dictionary
    
    """

    # load preprocessed sqad dataset
    with open(saved_dataset_file) as f: 
        data = json.load(f)
    print("Sqad dataset loaded from: " + saved_dataset_file)

    return data

In [ ]:
sqad = SqadDataset(sqad_dir="drive/MyDrive/data/cz_sqad/data/", save_dir="drive/MyDrive/data/test.json")
sqad.process_dataset(100, 150)


Sqad dataset has been processed to: drive/MyDrive/data/test.json


# Loading model


Choose model checkpoint

In [ ]:
# either 'mbert' or 'albert'
model_type = 'mbert'

if model_type == 'mbert':
  model_checkpoint = "./drive/MyDrive/mbert_models/bert_finetuned_czech_squad"
elif model_type == 'albert':
  model_checkpoint = "./drive/MyDrive/albert_models/albert_finetuned"
else:
  print("wrong model type name")

# albert xlarge pretrained on squad v2 for experiments
# model_checkpoint = "ktrapeznikov/albert-xlarge-v2-squad-v2"

Create reader



In [ ]:
reader = Reader(model_checkpoint, model_type=model_type)

Model loaded from: ./drive/MyDrive/mbert_models/bert_finetuned_czech_squad
Model has 177264386 parameters
Device selected:
cuda:0


Create retriever

In [ ]:
retriever = Retriever()

INFO:corpy.morphodita:Loading tagger.
2021-03-31 09:28:23.373 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]
INFO:deeppavlov.core.data.simple_vocab:[loading vocabulary from /root/.deeppavlov/models/ner_ontonotes_bert_mult/tag.dict]
2021-03-31 09:28:54.718 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model]
INFO:deeppavlov.core.models.tf_model:[loading model from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ontonotes_bert_mult/model


# Final pipeline

In [ ]:
def translate(question_cs, documents_cs, translator):
  """
  Translates the czech question and documents and returns
  question and list of documents in english

  """
  # we concatenate the question with all the documents to be translated at once
  # so we minimize the number of requests for googletrans
  delimiter = " _____ "
  concatenated = question_cs

  for doc in documents_cs:
    concatenated += delimiter
    concatenated += doc

  # and translate as a whole
  concatenated = translator.translate(concatenated, src='cs', dest='en').text
  # and split again
  delimiter = "_____"
  concatenated = concatenated.split(delimiter)
  
  # get translated question and doc
  question = concatenated[0]
  documents = concatenated[1:]

  return question, documents

In [ ]:
def find_answer(question, reader, retriever, translator, model_type):
  """
  Finds the answer to the question

  """

  question_cs = question # save czech question

  # wiki search
  documents_cs, article_list = retriever.retrieve(question)

  # for saving the best results
  bestAnswers = []
  bestDocs = []
  bestLogProbs = []
  bestSummedLogProbs = []
  
  # translate according to model type
  if model_type == 'mbert':
    question = question_cs
    documents = documents_cs

  elif model_type == 'albert':
    # translate question and documents for reader
    question, documents = translate(question_cs, documents_cs, translator)
  # delete null strings
  documents = [x for x in documents if len(x.strip())]

  # iterate over retrieved paragraphs
  for idx, document in enumerate(documents):
    # strip whitespaces
    document = document.strip()

    # chceck if any document has been found for the question
    if document == "":
      continue;

    #get answer -------------------------------------------
    answers = reader.get_answers(question, document)
    log_conf = 0
    log_conf_summed = 0

    # choose valid answer
    answer = ''
    for answer in answers:
      if answer['text'] != '':
        log_conf = answer['score']
        answer = answer['text']
        log_conf_summed = Retriever.count_log_conf(answer, answers)
        break
    #######################################################
    if type(answer) is not str:
      continue

    # save probs and answer
    bestAnswers.append(answer)
    bestLogProbs.append(log_conf)
    bestSummedLogProbs.append(log_conf_summed)
    # save retrieved doc
    bestDocs.append(documents_cs[idx])

  ############################################################
  # check if any answer was found
  if len(bestLogProbs) == 0 or bestAnswers[np.argmax(bestLogProbs, axis=0)] == '':
    return ""

  # get the best doc
  # get best answer from retriever according to reader
  document = bestDocs[np.argmax(bestLogProbs, axis=0)]
  answer = bestAnswers[np.argmax(bestLogProbs, axis=0)]

  # translate the final answer
  answer_en = answer
  if model_type == 'albert':
    answer =  translator.translate(answer, src='en', dest='cs').text

  return answer, answer_en, document, bestAnswers, bestLogProbs, bestSummedLogProbs, article_list

# Evaluation on  sqad

In [ ]:
def sqad_eval(from_q, to_q, data, save_results_to, model_type):
  """
  Evaluates the model on sqad dataset
  """
  # create translator
  translator = Translator()

  # write results to
  f = open(save_results_to, "w")

  # for counting correct answers
  score = 0

  for i in tqdm(range(from_q, to_q+1)):

    # get from dataset
    question = data[str(i)]["question"]
    correct_answer = data[str(i)]["answer"]
    lemmatized_answer = data[str(i)]["answer_lemma"]

    # get answer and other info to the specific question
    answer, answer_en, document, bestAnswers, bestLogProbs, bestSummedLogProbs, article_list = find_answer(
        question, reader, retriever, translator, model_type)

    # write the result to file
    f.write("----------------------------------------------------------------\n"+
            "-----------------------| otázka č." + str(i) + " |------------------------\n" +
            "----------------------------------------------------------------\n"+
            "::otázka : " + question + "\n" +
            "::odpověď: " + answer + " / " + answer_en + "\n" + 
            "::správná odpověď podle sqad : " + correct_answer + "\n" +
            "::lemma odpovědi podle sqad : " + lemmatized_answer + "\n\n" +
            "----------------------------------------------------------------\n"+
            "získaný dokument: " + document + 
            "\n----------------------------------------------------------------\n")
    # what answers did we get over-all - debugging info
    for listitem in bestAnswers:
      f.write('%s ;; ' % listitem)
    f.write("\n")
    for listitem in bestLogProbs:
      f.write('%s ;; ' % listitem)
    f.write("\n")
    for listitem in bestSummedLogProbs:
      f.write('%s ;; ' % listitem)
    f.write("\n")
    for listitem in article_list:
      f.write('%s ;; ' % listitem)
    f.write("\n\n\n")

    # convert to lowercase
    answer = answer.lower()
    answer_en = answer_en.lower()
    correct_answer = correct_answer.lower()
    lemmatized_answer = lemmatized_answer.lower()
    # increment score, if we got any match between the original and retrieved answer
    if (answer in correct_answer or correct_answer in answer or
        answer in lemmatized_answer or lemmatized_answer in answer or 
        answer_en in lemmatized_answer or lemmatized_answer in answer_en):
      score += 1

  # close file descriptor
  f.close()

  # get the count of questions answered for score calculation
  answered_count = to_q - from_q + 1

  print("done")
  print("score: " + str(score) + "/" + str(answered_count))

In [ ]:
# load preprocessed sqad dataset
data = SqadDataset.load_sqad("drive/MyDrive/data/sqad_processed_without_yes_no.json")

Sqad dataset loaded from: drive/MyDrive/data/sqad_processed_without_yes_no.json


Run the evaluation and save the results to the chosen file

In [ ]:
save_to = "drive/MyDrive/data/saved_answers/saved_answers_mbert/test_101-200_pavlov+morphodita.txt"

sqad_eval(101, 200, data, save_to, model_type)


done
score: 68/100


# Testing NER

Build model - if you don't have the model stored, use 'download=True'

In [ ]:
# Download and load model (set download=False to skip download phase)
ner = build_model(configs.ner.ner_ontonotes_bert_mult, download=False)

Testing

In [ ]:
named_entities = get_pavlov_named_entities("kdo byl Albert einstein?")
print(named_entities)

['Albert einstein']


In [ ]:
def get_pavlov_named_entities(question):
    """
    Extracts named entities from the question.

    """
    ner_tags = ner([question])

    named_entities = ""
    for idx, tag in enumerate(ner_tags[1][0]):
      if tag != 'O':
        named_entities += ner_tags[0][0][idx] + " "

    NE = []
    if len(named_entities.strip()) != 0:
      NE.append(named_entities.strip())

    return NE

# Testing MorphoDiTa

In [ ]:
from corpy.morphodita import Tagger
tagger = Tagger("./czech-morfflex-pdt-161115/czech-morfflex-pdt-161115.tagger")

INFO:corpy.morphodita:Loading tagger.


In [ ]:
sentence = "Perak (známý také pod arabským názvem Darul Ridzuan) je sultanát a jeden z federálních států Malajsie.Kedah se nachází v západní části Malajsie na pobřeží Malackého průlivu."

In [ ]:
text = sentence.lower()
text = re.split("\W", text)
text = (" ").join(text)

tokens = list(tagger.tag(text, convert='strip_lemma_id'))

lemmas = []
for token in tokens:
  lemmas.append(token.lemma)

print(lemmas)

['perak', 'známý', 'také', 'pod', 'arabský', 'název', 'darul', 'ridzuan', 'být', 'sultanát', 'a', 'jeden', 'z', 'federální', 'stát', 'malajsie', 'kedah', 'se', 'nacházet', 'v', 'západní', 'část', 'malajsie', 'na', 'pobřeží', 'malacký', 'průliv']


# Ask a question

In [ ]:
# create translator
translator = Translator()

question = "Kdy skončila druhá světová válka?"
answer = find_answer(question, reader, retriever, translator, model_type)

if len(answer) == 0:
  print(answer)
else:
  print(question)
  print()
  print(answer[0])
  if model_type == 'albert':
    print(answer[1])
  print()
  #print(answer[1])
  print(answer[2])
  print(answer)